<a href="https://colab.research.google.com/github/StSahtouris/StSahtouris/blob/main/copy_of_threat_hunting_new_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#εγκατάσταση πακέτου pycaret (https://pycaret.org/)
!pip install pycaret
#εγκατάσταση πακέτου dataprep (https://dataprep.ai/)
!pip install dataprep
#***ΠΡΟΣΟΧΗ*** μετά την εγκατάσταση χρειάζεται restart το session
#***θα σας ζητηθεί επανεκκίνηση με σχετικό κόκκινο μήνυμα μετά την ολοκλήρωση της εγκατάστασης
#***Πατήστε το κουμπί RESTART SESSION στο σχετικό μήνυμα

In [ ]:
#εισαγωγή βιβλιοθηκών - πακέτων
import pandas as pd
import matplotlib.pyplot as plt
from pycaret.regression import *
from dataprep.eda import create_report
from pyod.models.iforest import IForest

In [ ]:
#εισαγωγή του συνόλου δεδομένων
url = "https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/Network-flows.csv"
try:
    df = pd.read_csv(url) # Indent this line to be part of the 'try' block
    print(f"Σχήμα δεδομένων: {df.shape}")
except Exception as e:
    print(f"Error: {e}")

Σχήμα δεδομένων: (155123, 19)


In [ ]:
#ανακάτεμα των δεδομένων
if len(df) > 100000:
  print
df = df.sample(frac = 1, random_state=42)

In [ ]:
#οπτικοποίηση συνόλου δεδομένων
df.head()

,src_ip,src_port,dst_ip,dst_port,proto,pktTotalCount,octetTotalCount,min_ps,max_ps,avg_ps,std_dev_ps,flowStart,flowEnd,min_piat,max_piat,avg_piat,std_dev_piat,flowEndReason,flowDuration
24968,192.168.121.80,56130,172.16.255.200,53,17,2,138,61,77,69.0,8.0,1.559770e+09,1.559770e+09,0.000454,0.000454,0.000454,0.0,2,0.000454
122624,192.168.128.58,6881,181.49.95.58,54276,17,2,264,132,132,132.0,0.0,1.559766e+09,1.559766e+09,82.811775,82.811775,82.811775,0.0,2,82.811775
150888,192.168.128.134,61181,172.16.255.183,53,17,2,184,78,106,92.0,14.0,1.559765e+09,1.559765e+09,0.154224,0.154224,0.154224,0.0,2,0.154224
155036,192.168.128.151,58052,172.16.255.200,53,17,2,356,60,296,178.0,118.0,1.559771e+09,1.559771e+09,0.000352,0.000352,0.000352,0.0,2,0.000352
15558,192.168.128.251,56494,172.16.255.200,53,17,2,295,74,221,147.5,73.5,1.559683e+09,1.559683e+09,0.064521,0.064521,0.064521,0.0,2,0.064521


In [ ]:
#Διερευνητική ανάλυση δεδομένων (Exploratory Data Analysis - EDA)
if len(df) > 100000:
  print
  sampled_df = df.sample(n=100000, random_state=42)
  create_report(sampled_df)
else:
  create_report(df) # Indented this line to be part of the 'else' block

In [ ]:

#διαχωρισμός των δεδομένων σε σύνολα εκπαίδευσης (train) και δοκιμών (test)
train_size = int(0.8 * len(df))
train_df = df[:train_size]
test_df = df[train_size:]

In [ ]:
#Εκκίνηση της ρύθμισης του PyCaret με προεπεξεργασία δεδομένων
reg_setup = setup(data=df, target='flowDuration', session_id=42,
                  normalize=True, transformation=True,
                  ignore_features=['proto'])

,Description,Value
0,Session id,42
1,Target,flowDuration
2,Target type,Regression
3,Original data shape,"(155123, 19)"
4,Transformed data shape,"(155123, 18)"
5,Transformed train set shape,"(108586, 18)"
6,Transformed test set shape,"(46537, 18)"
7,Ignore features,1
8,Numeric features,15
9,Categorical features,2


In [ ]:
#σύγκριση των διαθέσιμων μοντέλων ΜΜ
best = compare_models(cross_validation=False, n_select = 1)
%load_ext tensorboard

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,5.5872,506.7939,22.5121,0.9878,0.2226,290.7427,5.2700
et,Extra Trees Regressor,3.5083,634.4177,25.1877,0.9847,0.0621,0.0261,59.0300
xgboost,Extreme Gradient Boosting,5.5249,643.3672,25.3647,0.9845,0.2221,201.2009,4.3100
br,Bayesian Ridge,13.1667,795.0469,28.1966,0.9808,1.0917,2322.6405,4.3900
lr,Linear Regression,13.1666,795.0466,28.1966,0.9808,1.0917,2322.2514,3.1200
rf,Random Forest Regressor,4.0887,851.8136,29.1858,0.9794,0.0718,0.0310,186.2200
par,Passive Aggressive Regressor,18.2600,922.7061,30.3761,0.9777,1.7204,15175.9317,15.9100
gbr,Gradient Boosting Regressor,12.5026,1632.4546,40.4036,0.9606,0.6497,3284.5445,56.3400
dt,Decision Tree Regressor,5.9945,1989.6609,44.6056,0.9519,0.1030,0.0426,7.5800
ridge,Ridge Regression,34.8809,4263.7085,65.2971,0.8970,2.1834,39364.2183,3.1400


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
#αναφορά των υπερπαραμέτρων του μοντέλου με την υψηλότερη απόδοση
print(best)

LGBMRegressor(n_jobs=-1, random_state=42)


In [ ]:
#μέτρα αξιολόγησης του μοντέλου με την υψηλότερη απόδοση
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
#δημιουργία μοντέλου ανίχνευσης ανωμαλιών Isolation Forest
iforest = IForest(contamination=0.1)
iforest.fit(train_df[['flowDuration']])


IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
    verbose=0)

In [ ]:
#πίνακας προβλέψεων με ανωμαλίες στο σύνολο δοκιμών
test_df['anomaly'] = iforest.predict(test_df[['flowDuration']])

In [ ]:
#δημιουργία νέας στήλης 'Label' με βάση το αποτέλεσμα του ελέγχου ανίχνευσης ανωμαλιών
test_df['Label'] = test_df['anomaly'].apply(lambda x: 'Anomaly' if x == 1 else 'Normal')

In [ ]:
#εκτύπωση των ανωμαλιών
print("Ανωμαλίες στο Test Set:")
print(test_df[test_df['anomaly'] == 1][['flowDuration', 'Label']])

Ανωμαλίες στο Test Set:
        flowDuration    Label
48087      82.477634  Anomaly
103270   1691.979700  Anomaly
69126     440.424092  Anomaly
38319    1682.968303  Anomaly
99902     248.259878  Anomaly
...              ...      ...
123855    409.222841  Anomaly
122537    184.168771  Anomaly
41090     270.460443  Anomaly
87498      93.203994  Anomaly
110268    245.606294  Anomaly

[3052 rows x 2 columns]


In [ ]:
#κατανομή των προβλέψεων για το σύνολο δοκιμών
total_test_samples = len(test_df)
normal_test_samples = len(test_df[test_df['Label'] == 'Normal'])
anomaly_test_samples = len(test_df[test_df['Label'] == 'Anomaly'])

print(f"\nΣύνολο Δοκιμών (Test Set): {total_test_samples}")
print(f"Φυσιολογικές Προβλέψεις: {normal_test_samples}")
print(f"Ανώμαλες Προβλέψεις: {anomaly_test_samples}")
print(f"Ποσοστό Ανωμαλιών: {(anomaly_test_samples / total_test_samples) * 100:.2f}%")


Σύνολο Δοκιμών (Test Set): 31025
Φυσιολογικές Προβλέψεις: 27973
Ανώμαλες Προβλέψεις: 3052
Ποσοστό Ανωμαλιών: 9.84%
